In [8]:
import pandas as pd
import re
from tqdm import tqdm
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

In [9]:
import pandas as pd

# Load the CSV dataset into a Pandas DataFrame
df = pd.read_csv("DatasetKabupaten/Skripsi/kabupaten wonosobo.csv")

# Filter the DataFrame to include only rows where the ticket status is "Selesai"
# and the department is one of the specific three departments
specific_departments = [
    "Kepolisian Daerah Jawa Tengah",
    "DINAS PEKERJAAN UMUM BINA MARGA CIPTA KARYA",
    "DINAS KESEHATAN"
]
filtered_df = df[
    (df["Status"] == "Selesai") & (df["Departement"].isin(specific_departments))
]

# Keep only the 'Message', 'Status', and 'Departement' columns
filtered_df = filtered_df[["Message", "Status", "Departement"]]

# Save the preprocessed data to a new CSV file
filtered_df.to_csv("wonosobo_filtered.csv", index=False)

# Display the preprocessed DataFrame
print(filtered_df)

                                                Message   Status  \
0     Assalamualaikum...Pak gubernur.. Saya mau usul...  Selesai   
1     Pak Gub,,, biaya perpanjangan SIM A umum,B1, B...  Selesai   
3     Kpd Yth Gub Jateng. Gini pak Ganjar, akses jal...  Selesai   
4                         JALAN WONOSOBO-SEMARANG RUSAK  Selesai   
17    LAPOR pak gub, mohon d tindak lanjuti playanan...  Selesai   
...                                                 ...      ...   
1493  Mohon bantuannya pak.. Anak kami lahir prematu...  Selesai   
1574  Alamat: Kabupaten/Kota wonosobo, Kecamatan sap...  Selesai   
1584  lokasi sapuran wonosobo..saya habis bayar paja...  Selesai   
1586  Alamat: Kabupaten/Kota Wonosobo, Kecamatan Sap...  Selesai   
1598  Alamat: Kabupaten/Kota Wonosobo, Kecamatan Kal...  Selesai   

                                      Departement  
0     DINAS PEKERJAAN UMUM BINA MARGA CIPTA KARYA  
1                   Kepolisian Daerah Jawa Tengah  
3     DINAS PEKERJAAN UMUM 

In [10]:
# Load the dataset
df = pd.read_csv("wonosobo_filtered.csv")


def preprocess_label(text):
    # Convert text to lowercase
    text = str(text).lower()
    return text


# Define a function to preprocess text
def preprocess_text(text):
    # Convert text to lowercase
    text = str(text).lower()

    # Remove special characters, numbers, and punctuation
    text = re.sub(r"[^a-z\s]", " ", text)

    # Tokenize the text
    tokens = text.split()

    # Remove stopwords
    stopword_factory = StopWordRemoverFactory()
    stopword_remover = stopword_factory.create_stop_word_remover()
    tokens = [stopword_remover.remove(word) for word in tokens]

    """# Stemming
    stemmer_factory = StemmerFactory()
    stemmer = stemmer_factory.create_stemmer()
    tokens = [stemmer.stem(word) for word in tokens]"""

    # Join the tokens back into a single string
    preprocessed_text = " ".join(tokens)
    if preprocessed_text and preprocessed_text[0].isspace():
        preprocessed_text = preprocessed_text[1:]

    preprocessed_text = re.sub(r"\s+", " ", text)
    # print(preprocessed_text)
    return preprocessed_text

In [11]:
#3152
print(df['Message'][50])

proyek betonisasi jalan ngalian kalidadap tidak sesuai harapan..proyek milyaran pelaksanaan abur radul..akses jalan ditutup total selama 5 bulan..kasian pengguna sepeda motor tidak bisa lewat..


In [12]:
tqdm.pandas(desc="Preprocessing")
df["Message"] = df["Message"].progress_apply(preprocess_text)
df["Departement"] = df["Departement"].progress_apply(preprocess_label)


# Save the preprocessed dataset to a new CSV file
df.to_csv("wonosobo_filtered.csv", index=False)

Preprocessing: 100%|██████████| 159/159 [00:00<00:00, 115100.85it/s]


In [13]:
# Apply the preprocessing function to the text column
df['cleaned_text'] = df['Message'][:10].apply(preprocess_text)

# Save the preprocessed dataset to a new CSV file
df.to_csv('preprocessed_dataset.csv', index=False)

In [22]:
print(df['Message'][:10])

0    assalamualaikum pak gubernur saya mau usul mbo...
1    pak gub biaya perpanjangan sim a umum b b umum...
2    kpd yth gub jateng gini pak ganjar akses jalan...
3                        jalan wonosobo semarang rusak
4    jalan rusak parah silento wonosobo perbatasan ...
5    assalamualaikum saya mau melaporkan tentang ja...
6    lapor nama sy helmy jalan didepan rmh sy rusak...
7    yang terhormat bp gubernur kami masyarakat ds ...
8    yth bp ganjar sya mau menanyakan hasil ott di ...
9    pak ganjar jalan lintas provinsi wonosobo jawa...
Name: Message, dtype: object


In [ ]:
# Assuming df is your DataFrame containing the cleaned text
for cleaned_text in df['cleaned_text']:
    print(cleaned_text)
